In [15]:
import pandas as pd
from pathlib import Path

In [ ]:
DATA_PATH = Path("data_out/ma2018_plan_county_year_inner.csv")
if not DATA_PATH.exists():
    raise FileNotFoundError("Run hwk1_code.ipynb first to create data_out/ma2018_plan_county_year_inner.csv")

ma2018 = pd.read_csv(DATA_PATH, low_memory=False)

# 2) Basic cleaning / types
ma2018 = ma2018.dropna(subset=["contractid", "planid", "plan_type"]).copy()
ma2018["planid_num"] = pd.to_numeric(ma2018["planid"], errors="coerce")
ma2018["avg_enrollment"] = pd.to_numeric(ma2018["avg_enrollment"], errors="coerce")

# define a "plan" as contractid + planid (same plan can appear in many counties)
ma2018["plan_key"] = ma2018["contractid"].astype(str) + "-" + ma2018["planid_num"].astype("Int64").astype(str)